In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul 28 09:52:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   35C    P8              16W /  72W |      0MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%capture
%pip install datasets>=1.18.3
%pip install transformers==4.11.3
%pip install torch
%pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
%pip install jiwer
%pip install accelerate -U
%pip install python-dotenv
%pip install librosa datasets

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

True

In [4]:
from huggingface_hub import notebook_login
import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder

import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder
token=os.environ.get("HF_TOKEN")
#huggingface_hub.login(token=token,add_to_git_credential=True, new_session=False, write_permission=True)

api=HfApi(token=token)
folder = HfFolder()
folder.save_token(token)



# huggingface_hub.login(token=os.environ.get("HF_TOKEN"),add_to_git_credential=True, new_session=False, write_permission=True)

In [5]:
%%capture
%apt install git-lfs

TORGO

In [6]:
# import pandas as pd
# import os
# import librosa

# def data_generator(data_dir):
#     metadata_path = os.path.join(data_dir, 'metadata.csv')
#     metadata = pd.read_csv(metadata_path)

#     for _, row in metadata.iterrows():
#         file_path = os.path.join(data_dir, row['file_name'])
#         transcription = row['transcription']
#          # Load the audio file with librosa
#         audio_data, sampling_rate = librosa.load(file_path, sr=16_000)  # Ensure the sampling rate is 16 kHz

#         # Structure the audio data and sampling rate in a dictionary
#         audio_dict = {"array": audio_data, "sampling_rate": sampling_rate}

#         yield {"audio": audio_dict, "path": file_path, "transcription": transcription}


# from datasets import Dataset, Features, Audio, Value

# data_dir = '/content/drive/MyDrive/TORGO-big'  # Adjust this path as needed

# features = Features({
#     "path": Value("string"),
#     "audio": Audio(sampling_rate=16_000),
#     "transcription": Value("string")
# })

# dataset = Dataset.from_generator(
#     generator=data_generator,
#     features=features,
#     gen_kwargs={"data_dir": data_dir}
# )


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)

# dataset = dataset.train_test_split(test_size=0.2)


UASPEECH

In [7]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [8]:
# test if pyarrow works
import pyarrow
print(pyarrow.concat_tables)

<cyfunction concat_tables at 0x7fc8a04bd970>


In [9]:
import sys
sys.executable

'/home/zeus/miniconda3/envs/cloudspace/bin/python'

In [10]:
# import pandas as pd
# import os
# import librosa
# from datasets import Dataset

# def load_parquet_directory(directory_path, use_narrow_data=False):
#     all_dfs = []
#     for file_name in os.listdir(directory_path):
#         if file_name.endswith('.parquet'):
#             file_path = os.path.join(directory_path, file_name)
#             df = pd.read_parquet(file_path)
#             print("parquet read")
#             all_dfs.append(df)
#             if use_narrow_data:
#               break

#     return pd.concat(all_dfs, ignore_index=True)

# def convert_dataframe_to_dataset(dataframe):
#     return Dataset.from_pandas(dataframe)


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)
# directory_path = '/teamspace/uploads/uaspeechall/data' # Update this path
# df = load_parquet_directory(directory_path, use_narrow_data=False)

# print("parquet loaded")
# dataset = convert_dataframe_to_dataset(df)

###### OTHER WAY


# import pyarrow.parquet as pq
# from datasets import Dataset
# import glob

# def load_memory_mapped_dataset(directory_path):
#     # Get all parquet files in the directory
#     parquet_files = glob.glob(f'{directory_path}/*.parquet')
    
#     # Read the parquet files using memory mapping
#     tables = [pq.read_table(file, memory_map=True) for file in parquet_files]
    
#     # Concatenate all tables
#     combined_table = pyarrow.concat_tables(tables)
    
#     # Convert to a Dataset object
#     return Dataset(pyarrow.Table.from_batches(combined_table.to_batches()))

# # Load the dataset
# dataset = load_memory_mapped_dataset('/teamspace/uploads/uaspeechall/data')

####### YET ANOTHER WAY
import pyarrow.parquet as pq
from datasets import Dataset, concatenate_datasets
import glob
import pyarrow as pa

def chunk_generator(table, chunk_size=10000):
    num_rows = table.num_rows
    for i in range(0, num_rows, chunk_size):
        yield table.slice(i, min(chunk_size, num_rows - i)).to_batches()[0]

def process_parquet_files(directory_path, chunk_size=10000):
    parquet_files = glob.glob(f'{directory_path}/*.parquet')
    for file in parquet_files:
        print(f"Processing file: {file}")
        table = pq.read_table(file, memory_map=True)
        for chunk in chunk_generator(table, chunk_size):
            yield Dataset(pa.Table.from_batches([chunk]))

# Usage
directory_path = '/teamspace/uploads/uaspeechall/data'
all_datasets = []

for chunk_dataset in process_parquet_files(directory_path):
    # Process each chunk here
    # For example, you can apply your transformations:
    # transformed_chunk = chunk_dataset.map(your_transform_function)
    all_datasets.append(chunk_dataset)
# Combine all chunks into a single dataset
dataset = concatenate_datasets(all_datasets)


Processing file: /teamspace/uploads/uaspeechall/data/test-00000-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00001-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00002-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00003-of-00007.parquet


In [ ]:
dataset

Dataset({
    features: ['audio', 'speaker_id', 'transcription'],
    num_rows: 2500
})

In [ ]:
!pip install torchvision torchaudio pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 17.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Atte

In [ ]:
print(dataset[0]["audio"].keys())

dict_keys(['path', 'array', 'sampling_rate'])


In [ ]:
# Perhaps not needed at all

import numpy as np

def convert_audio_format(batch):
    audio_data = batch["audio"]
    
    # The audio data is already in the correct format
    audio_array = np.array(audio_data["array"]).astype(np.float32)
    sampling_rate = audio_data["sampling_rate"]

    # Update the audio dictionary (this step might be unnecessary if the data is already in the correct format)
    batch["audio"] = {"array": audio_array, "sampling_rate": sampling_rate}
    return batch

# Apply the conversion function to the dataset
dataset = dataset.map(convert_audio_format)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
from datasets import load_dataset, load_metric, Audio

train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
train_dataset[0]

{'audio': {'path': None,
  'array': array([-6.10351562e-05, -9.15527344e-05, -6.10351562e-05, ...,
         -3.05175781e-05,  0.00000000e+00,  0.00000000e+00]),
  'sampling_rate': 16000},
 'speaker_id': 'M07_B2_UW11_M7',
 'transcription': 'supreme'}

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import soundfile as sf
# import numpy as np

# def save_audio_samples(dataset, num_samples=5):
#     for i in range(num_samples):
#         index = np.random.randint(0, len(dataset))
#         example = dataset[index]
#         audio_data = example["audio"]
        
#         if isinstance(audio_data, dict) and "array" in audio_data and "sampling_rate" in audio_data:
#             audio_array = audio_data["array"]
#             sampling_rate = audio_data["sampling_rate"]
            
#             filename = f"audio_sample_{i}.wav"
#             sf.write(filename, audio_array, sampling_rate)
#             print(f"Saved {filename}")
#             print(f"Transcription: {example['transcription']}")
#             print(f"Speaker ID: {example['speaker_id']}")
#             print("="*50)

# # Use the function
# save_audio_samples(train_dataset, num_samples=5)

Saved audio_sample_0.wav
Transcription: Sierra
Speaker ID: M07_B2_LS_M6
Saved audio_sample_1.wav
Transcription: One
Speaker ID: F05_B2_D1_M8
Saved audio_sample_2.wav
Transcription: Nuremberg
Speaker ID: M12_B2_UW3_M5
Saved audio_sample_3.wav
Transcription: her
Speaker ID: F04_B1_CW61_M3
Saved audio_sample_4.wav
Transcription: hanger
Speaker ID: M10_B3_UW7_M8


In [ ]:
import numpy as np
import soundfile as sf
from IPython.display import display, Audio, HTML
import os

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    for i in range(num_examples):
        index = np.random.randint(0, len(dataset))
        example = dataset[index]
        
        print(f"Example {i+1}:")
        print(f"Transcription: {example['transcription']}")
        print(f"Speaker ID: {example['speaker_id']}")
        
        audio_data = example["audio"]
        if isinstance(audio_data, dict) and "array" in audio_data and "sampling_rate" in audio_data:
            audio_array = audio_data["array"]
            sampling_rate = audio_data["sampling_rate"]
            
            print(f"Audio length: {len(audio_array) / sampling_rate:.2f} seconds")
            
            # Save as WAV file
            filename = f"audio_sample_{i}.wav"
            sf.write(filename, audio_array, sampling_rate)
            print(f"Audio saved as: {filename}")
            
            # Display audio player (this may or may not work depending on your environment)
            display(Audio(data=audio_array, rate=sampling_rate))
            
            # Provide a download link
            file_path = os.path.abspath(filename)
            display(HTML(f'<a href="file://{file_path}" download="{filename}">Download {filename}</a>'))
        
        else:
            print("Audio: Unexpected format")
        
        print("\n" + "="*50 + "\n")

# Use the function
show_random_elements(train_dataset, num_examples=1)

Example 1:
Transcription: Delete
Speaker ID: F02_B1_C3_M6
Audio length: 4.26 seconds
Audio saved as: audio_sample_0.wav




Example 2:
Transcription: bosom
Speaker ID: M09_B2_UW54_M4
Audio length: 2.97 seconds
Audio saved as: audio_sample_1.wav




Example 3:
Transcription: we
Speaker ID: F04_B3_CW35_M2
Audio length: 2.06 seconds
Audio saved as: audio_sample_2.wav




Example 4:
Transcription: Whiskey
Speaker ID: M07_B2_LW_M2
Audio length: 2.32 seconds
Audio saved as: audio_sample_3.wav




Example 5:
Transcription: thumb
Speaker ID: M07_B2_UW100_M8
Audio length: 1.90 seconds
Audio saved as: audio_sample_4.wav


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

In [ ]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 10.5 MB/s eta 0:00:00
  Using cached regex-2024.7.24-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 112.9 MB/s eta 0:00:0000:0100:01
Using cached regex-2024.7.24-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (776 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 104.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-large-960h-lv60-self-dysarthria"
user_name = "jmaczan"

In [ ]:
tokenizer.push_to_hub(repo_name, token=token)

CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-960h-lv60-self-dysarthria/commit/00944e0598939c19919f9ea3c54317c76b5d0fbf', commit_message='Upload tokenizer', commit_description='', oid='00944e0598939c19919f9ea3c54317c76b5d0fbf', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

RuntimeError: Failed to import transformers.models.wav2vec2.processing_wav2vec2 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
train_dataset[0]["audio"]

{'array': [-0.000213623046875,
  -0.000579833984375,
  -0.000640869140625,
  -0.00067138671875,
  -0.000640869140625,
  -0.00042724609375,
  -0.00054931640625,
  -0.00048828125,
  -0.0003662109375,
  -0.000213623046875,
  -0.00030517578125,
  -0.0003662109375,
  -6.103515625e-05,
  -0.0001220703125,
  -6.103515625e-05,
  -6.103515625e-05,
  3.0517578125e-05,
  0.0,
  6.103515625e-05,
  0.00030517578125,
  0.00018310546875,
  0.0001220703125,
  0.000244140625,
  0.000457763671875,
  0.000396728515625,
  0.0006103515625,
  0.000579833984375,
  0.0006103515625,
  0.001922607421875,
  0.002166748046875,
  0.001617431640625,
  0.0015869140625,
  0.001434326171875,
  0.001312255859375,
  0.001251220703125,
  0.0008544921875,
  0.000457763671875,
  0.00054931640625,
  0.00018310546875,
  0.0001220703125,
  -9.1552734375e-05,
  -0.000274658203125,
  -0.00054931640625,
  -0.00177001953125,
  -0.002105712890625,
  -0.00238037109375,
  -0.00238037109375,
  -0.0020751953125,
  -0.002471923828125,


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset)-1)

print(train_dataset[rand_int]["transcription"])
ipd.Audio(data=train_dataset[rand_int]["audio"]["array"], autoplay=False, rate=16000)

finger


In [ ]:
rand_int = random.randint(0, len(train_dataset)-1)

print("Target text:", train_dataset[rand_int]["transcription"])
print("Input array shape:", train_dataset[rand_int]["audio"]["array"])#.shape)
print("Sampling rate:", train_dataset[rand_int]["audio"]["sampling_rate"])

Target text: agricultural
Input array shape: [-0.0001220703125, -0.00018310546875, -0.000213623046875, -0.000213623046875, -0.000152587890625, -0.000152587890625, -0.000213623046875, -0.000335693359375, -0.00042724609375, -0.000396728515625, -0.00030517578125, -0.000274658203125, -0.000244140625, -0.000213623046875, -0.000244140625, -0.000152587890625, 3.0517578125e-05, 6.103515625e-05, 0.0, 0.00018310546875, 0.000213623046875, 0.00018310546875, 0.0003662109375, 0.000244140625, 0.00018310546875, 0.000244140625, 0.00018310546875, 6.103515625e-05, 0.00018310546875, 0.00018310546875, 6.103515625e-05, 3.0517578125e-05, -3.0517578125e-05, -3.0517578125e-05, 9.1552734375e-05, 0.000213623046875, 0.0003662109375, 0.000579833984375, 0.00048828125, 0.00048828125, 0.000640869140625, 0.00048828125, 0.000640869140625, 0.000823974609375, 0.0009765625, 0.000640869140625, 0.0009765625, 0.001007080078125, 0.000457763671875, 0.000518798828125, 0.000823974609375, 0.00103759765625, 0.000732421875, 0.00051

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [ ]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/737 [00:00<?, ? examples/s]

### Training


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-48-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
!pip install -U flash-attn --no-build-isolation

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h-lv60-self",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.1,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
)

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=20,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

### Training

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,16.998000,3.420462,1.000000
800,3.650700,3.281910,1.000000
1200,3.214800,3.023906,1.000000
1600,2.846400,2.581021,1.000000
2000,2.392300,2.236799,1.000000
2400,1.935800,1.707190,1.000000
2800,1.504300,1.343472,1.000000
3200,1.116900,0.897910,0.970149
3600,0.749000,0.576389,0.748982
4000,0.485500,0.287557,0.476255


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

Step,Training Loss,Validation Loss,Wer
400,16.998000,3.420462,1.000000
800,3.650700,3.281910,1.000000
1200,3.214800,3.023906,1.000000
1600,2.846400,2.581021,1.000000
2000,2.392300,2.236799,1.000000
2400,1.935800,1.707190,1.000000
2800,1.504300,1.343472,1.000000
3200,1.116900,0.897910,0.970149
3600,0.749000,0.576389,0.748982
4000,0.485500,0.287557,0.476255


In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1706542837.91637785477b.358.0:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-xls-r-300m-dysarthria/commit/7285fe71bbb98cd3409e79431e90c425a5a0b0c9', commit_message='End of training', commit_description='', oid='7285fe71bbb98cd3409e79431e90c425a5a0b0c9', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

### Evaluation

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_dict = processor(test_dataset[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
df = load_parquet_directory(directory_path, use_narrow_data=True)
dataset = convert_dataframe_to_dataset(df)

test_dataset_transcription = dataset.train_test_split(test_size=0.2)["test"]

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_dataset_transcription[0]["transcription"].lower())

Prediction:
watches

Reference:
no


### Optional 
Further training phase on next parquet file

In [ ]:
import pandas as pd
import os
import librosa
from datasets import Dataset

global_parquet_file_index = 1

def load_parquet_directory_by_file_index(directory_path, parquet_file_index=1):
    all_dfs = []
    current_parquet_file_index = 0
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.parquet'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_parquet(file_path)
            all_dfs.append(df)
            if current_parquet_file_index == parquet_file_index:
              break
            current_parquet_file_index = current_parquet_file_index + 1

    return pd.concat(all_dfs, ignore_index=True)

def convert_dataframe_to_dataset(dataframe):
    return Dataset.from_pandas(dataframe)


# def load_audio(batch):
#     audio_data, _ = librosa.load(batch['path'], sr=16000)
#     batch["audio"] = audio_data
#     return batch


# dataset = dataset.map(load_audio)
directory_path = '/content/drive/MyDrive/uaspeechall/data'
df = load_parquet_directory_by_file_index(directory_path, parquet_file_index=True)
dataset = convert_dataframe_to_dataset(df)




In [ ]:
import io
import numpy as np
from pydub import AudioSegment

def convert_audio_format(batch):
    # Extract the audio data from the dictionary
    audio_bytes = batch["audio"]["bytes"]  # Replace "data" with the correct key

    # Convert binary data to audio array
    audio_data = AudioSegment.from_file(io.BytesIO(audio_bytes), format="wav")
    # Convert to a NumPy array and normalize
    audio_array = np.array(audio_data.get_array_of_samples()).astype(np.float32) / 2**15

    # Update the audio dictionary
    batch["audio"] = {"array": audio_array, "sampling_rate": 16000}
    return batch

# Apply the conversion function to the dataset
dataset = dataset.map(convert_audio_format)

from datasets import load_dataset, load_metric, Audio

dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

tokenizer.push_to_hub(repo_name)

from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

model = Wav2Vec2ForCTC.from_pretrained(
    user_name+"/"+repo_name,
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()
trainer.push_to_hub()
